In [ ]:
# using Revise

using LinearAlgebra, QuadGK, Roots, FFTW, FastGaussQuadrature, SpecialFunctions
using Triangulate
using VlasovSolvers
import VlasovSolvers: advection!
import VlasovSolvers: samples, Particles, PF_step!, ParticleMover, kernel_poisson!, kernel_gyrokinetic!

using ProgressMeter, Printf
using Plots, LaTeXStrings

# Quadrature rules

In [ ]:
struct RectangleRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function RectangleRule(len, start, stop)
        points = LinRange(start, stop, len+1)[1:end-1]
        s = step(points) 
        weights = [s for _ = 1:len]
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct TrapezoidalRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function TrapezoidalRule(len, start, stop)
        points = LinRange(start, stop, len)[1:end]
        s = step(points) 
        weights = [s for _ = 1:len]
        weights[1] /= 2
        weights[end] /= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct SimpsonRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function SimpsonRule(len, start, stop)
        # make sure the number of points is uneven
        if len % 2 == 0
            len += 1
        end
        points = LinRange(start, stop, len)
        s = step(points) 
        weights = s/3 .* ones(len)
        weights[2:2:end-1] .*= 4
        weights[3:2:end-2] .*= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct GaussHermiteRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussHermiteRule(len, start, stop)
        points, weights = gausshermite(len)
        weights .*= exp.(points.^2)
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussLegendreRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussLegendreRule(len, start, stop)
        points, weights = gausslegendre(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussRadauRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussRadauRule(len, start, stop)
        points, weights = gaussradau(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussLobattoRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussLobattoRule(len, start, stop)
        points, weights = gausslobatto(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct KronrodRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function KronrodRule(len, start, stop)
        pts, w, _ = kronrod(len)
        weights = []
        points = []
        for i = 1:len
            push!(points, pts[i])
            push!(weights, w[i])
            push!(points, -pts[i])
            push!(weights, w[i])
        end
        push!(points, pts[end])
        push!(weights, w[end])
        
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        len = 2*len + 1
        new(len, start, stop, points, weights)
end
    end

In [ ]:
# Only works for rectangle and trapezoidal rules
function projection_onto_grid!(grid_dst, meshx, meshv, X, V_, W)
    meshxstep = meshx[2] - meshx[1]
    meshvstep = meshv[2] - meshv[1]
    grid_dst .= 0


    # Periodic Boundary conditions on velocity
    V = copy(V_)
    V[findall(v -> v >= meshv[end],  V)] .-= meshv[end] - meshv[1]
    V[findall(v -> v < meshv[1],  V)] .+= meshv[end] - meshv[1]
    
    for ipart = 1:length(X)
        idxgridx = Int64(fld(X[ipart],            meshxstep)) + 1
        idxgridv = Int64(fld(V[ipart] - meshv[1], meshvstep)) + 1
        idxgridxp1 = idxgridx<length(meshx) ? idxgridx+1 : 1
        idxgridvp1 = idxgridv<length(meshv) ? idxgridv+1 : 1
        
        tx = (X[ipart]             - (idxgridx-1) * meshxstep) / meshxstep
        tv = (V[ipart] - meshv[1]  - (idxgridv-1) * meshvstep) / meshvstep

        # println((idxgridv, V[ipart]))
        grid_dst[idxgridx  , idxgridv  ] += W[ipart] * (1-tx) * (1-tv)
        grid_dst[idxgridx  , idxgridvp1] += W[ipart] * (1-tx) * tv
        grid_dst[idxgridxp1, idxgridvp1] += W[ipart] * tx     * tv
        grid_dst[idxgridxp1, idxgridv  ] += W[ipart] * tx     * (1 - tv)
    end
end

# Numerical examples

In [ ]:
struct LandauDamping
    α
    kx
    μ
    β
    f0
    shortname
    longname
    L 
    vmin 
    vmax

    function LandauDamping(alpha, kx, mu, beta; 
                            shortname="Landau", longname="(Strong/Weak) Landau damping", L=nothing, vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        f(x,v) = (1 + alpha * cos(kx*x)) * exp(- beta * (v-mu)^2 / 2) / √(2π/beta)
        new(alpha, kx, mu, beta, f, shortname, longname, L, vmin, vmax)
    end
end

struct TwoStreamInstability 
    α
    kx
    β
    f0
    shortname
    longname
    L 
    vmin 
    vmax
    v0

    function TwoStreamInstability(alpha, kx, v0; 
                                    shortname="TSI", longname="Two-Stream Instability", L=nothing, vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        f(x,v) = (1 + alpha * cos(kx*x)) * (exp(- (v-v0)^2 / 2) + exp(- (v+v0)^2 / 2)) / (2*√(2π))
        new(alpha, kx, 1., f, shortname, longname, L, vmin, vmax, v0)
    end
end

struct TwoStreamInstabilityAlternativeFormulation
    α
    kx
    β
    f0
    shortname
    longname
    L 
    vmin 
    vmax

    function TwoStreamInstabilityAlternativeFormulation(alpha, kx; 
                                    shortname="TSI_alt", longname="Two-Stream Instability Alternative Formulation", L=nothing, vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        f(x,v) = (1 - alpha * cos(kx*x)) * exp(- v^2 / 2) * v^2 / √(2π)
        new(alpha, kx, 1., f, shortname, longname, L, vmin, vmax)
    end
end

struct BumpOnTail 
    α
    kx
    μ₁
    μ₂
    β₁
    β₂
    n₁
    n₂
    f0
    shortname
    longname
    L 
    vmin 
    vmax

    function BumpOnTail(alpha, kx, mu1, mu2, beta1, beta2; 
                        n1=0.9, n2=0.2, shortname="BoT", longname="Bump on Tail", L=nothing, vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        f(x,v) = (1 + alpha * cos(kx*x)) * ((n1*exp(-beta1*(v-mu1)^2 /2) + n2*exp(-beta2*(v-mu2)^2 / 2)) / √(2π))
        new(alpha, kx, mu1, mu2, beta1, beta2, n1, n2, f, shortname, longname, L, vmin, vmax)
    end
end

struct NonHomogeneousStationarySolution
    α
    kx
    β
    M₀
    f0
    shortname
    longname
    L
    vmin
    vmax

    function getM₀(α, β)
        find_zero( (M) -> M - α * √(2π/β) * besseli(1, M * β) * 2, 10)
        # 2 factor because of the definition of I₁(z) and C(t):
        # I₁(z) = 1/π ∫_0^π exp(z cos(θ)) cos(θ) dθ
        #       = 1/(2π) ∫_0^{2π} exp(z cos(θ)) cos(θ) dθ
        # C(t)  = 1/π ∫_0^{2π} ∫_{-∞}^{+∞} f(t,θ,v) cos(θ) dθ dv
        #       = 2α √(2π/β) I₁(βM₀)
    end

    function NonHomogeneousStationarySolution(alpha, kx, beta;
                                                shortname="non-homog", longname="Non Homogeneous Stationary Solution", L=nothing, 
                                                vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        m = getM₀(alpha, beta)
        new(alpha, kx, beta, m, (x,v) -> alpha * exp.(-beta * (v^2 / 2 - m * cos(x*kx))), shortname, longname, L, vmin, vmax)
    end
end

struct StationaryGaussian
    α
    kx
    β
    f0
    shortname
    longname
    L
    vmin
    vmax

    function StationaryGaussian(alpha, kx, beta;
                                                shortname="gaussian", longname="Stationary Gaussian", L=nothing, 
                                                vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        new(alpha, kx, beta, (x,v) -> alpha * exp.(-beta * v^2 / 2) / √(2π/beta), shortname, longname, L, vmin, vmax)
    end
end

struct Test
    α
    kx
    β
    f0
    shortname
    longname
    L
    vmin
    vmax

    function Test(alpha, kx, beta;
                                shortname="test", longname="Test", L=nothing, 
                                vmin=-9, vmax=9)
        isnothing(L) ? L = 2π / kx : nothing
        new(alpha, kx, beta, (x,v) -> alpha * exp.(-beta * (v - (vmin+vmax)/2)^2 / 2) / √(2π/beta) * exp.(-beta * (x - L/2)^2 / 2) / √(2π/beta), shortname, longname, L, vmin, vmax)
    end
end

example_landaudamping = LandauDamping(0.001, 0.5, 0., 1.; 
                                        longname="Weak Landau damping", shortname="weakLD", vmin=-9, vmax=9);
example_stronglandaudamping = LandauDamping(0.5, 0.5, 0., 1.; longname="Strong Landau damping", shortname="strongLD");
example_twostreaminstability = TwoStreamInstability(0.001, 0.2, 3.);
example_twostreaminstabilityalternativeformulation = TwoStreamInstabilityAlternativeFormulation(0.05, 0.2);
example_bumpontail = BumpOnTail(0.04, 0.3, 0, 4.5, 1, 4);
example_nonhomogeneousstationarysolution = NonHomogeneousStationarySolution(0.2, 1, 2);
example_stationarygaussian = StationaryGaussian(0.2, 1, 1);
example_test = Test(0.2, 1, 1);

# SL classique

In [ ]:
"""
    hmf_poisson!(fᵗ    :: Array{Complex{Float64},2},
                 mesh1 :: OneDGrid,
                 mesh2 :: OneDGrid,
                 ex    :: Array{Float64})

    Compute the electric hamiltonian mean field from the
    transposed distribution function

"""
function hmf_poisson!(fᵗ::Array{Complex{Float64},2},
        mesh1::OneDGrid,
        mesh2::OneDGrid,
        ex::Array{Float64}; K=1)

    n1 = mesh1.len
    rho = mesh2.step .* vec(sum(fᵗ, dims=1)) # ≈ ∫ f(t,x_i,v)dv, i=1, ..., n1
    kernel = zeros(Float64, n1)
    ker = -(mesh1.stop - mesh1.start) / (2π)
    for k=1:K
        kernel[1+k]   =  ker / k    # fourier mode  1
        kernel[end - (k-1)] = -ker / k    # fourier mode -1
    end
    ex .= real(ifft(fft(rho) .* 1im .* kernel))
end

function solve_SL!(nsteps, dt, f, mesh1, mesh2, kx; plotting=false::Bool)
    n1, n2 = size(f)
    fᵗ = zeros(Complex{Float64}, (n2,n1))
    transpose!(fᵗ, f)

    results = (Eelec = Array{Float64}(undef, nsteps),
                Etot = Array{Float64}(undef, nsteps),
                momentum = Array{Float64}(undef, nsteps),
                L²norm = Array{Float64}(undef, nsteps))


    ex = zeros(Float64, n1)
    hmf_poisson!(fᵗ, mesh1, mesh2, ex)
    advection!(fᵗ, mesh2, ex, 0.5dt)

    progression = ProgressMeter.Progress(nsteps,desc="Loop in time: ", showspeed=true)
    
    animation = @animate for istep = 1:nsteps
        results.Eelec[istep] = sum(ex.^2) * mesh1.step
        results.Etot[istep] = (results.Eelec[istep] + sum(mesh2.points'.^2 .* real(f)) * mesh1.step * mesh2.step) / 2
        results.momentum[istep] = sum(sum(real(f), dims=1) .* mesh2.points) * mesh1.step * mesh2.step
        results.L²norm[istep] = sum(real(f).^2) * mesh1.step * mesh2.step
    
        advection!(f, mesh1, mesh2.points, dt)
        transpose!(fᵗ, f)
        hmf_poisson!(fᵗ, mesh1, mesh2, ex)
        advection!(fᵗ, mesh2, ex, dt)
        transpose!(f, fᵗ) 
        
        if plotting
            plot(mesh1.points, mesh2.points, real(f)', size=(500, 500), st=:surface, camera=(0, 90))
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end
        
        ProgressMeter.next!(progression)
    end when plotting
    if !plotting
        animation = nothing
    end
    
    results.Eelec .= sqrt.(results.Eelec)
    results.Etot .= sqrt.(results.Etot)
    results.L²norm .= sqrt.(results.L²norm)
    
    return results, animation
end

# PIC solver

In [ ]:
function solve_PF!(nsteps, dt, particles, meshx, example, weights; plotting=false::Bool, kernel=kernel_poisson!, T=NaN)
    init_pos = copy(particles.x)
    init_vel = copy(particles.v)

    np = particles.nbpart

    results = (Eelec = Array{Float64}(undef, nsteps),
                Etot = Array{Float64}(undef, nsteps),
                momentum = Array{Float64}(undef, nsteps),
                L²norm = Array{Float64}(undef, nsteps),
                C = Array{Float64}(undef, nsteps),
                S = Array{Float64}(undef, nsteps),
                historyX = Array{Float64}(undef, np, nsteps+1),
                historyV = Array{Float64}(undef, np, nsteps+1),
                max_area_triangles = zeros(Float64, nsteps),
                mean_area_triangles = zeros(Float64, nsteps),
                min_area_triangles = zeros(Float64, nsteps))

    pmover = ParticleMover(particles, meshx, 1, dt; example.kx)

    results.historyX[:, 1] .= particles.x
    results.historyV[:, 1] .= particles.v

    if plotting
        widthx = -(-)(extrema(quadrulex.points)...)
        widthv = -(-)(extrema(quadrulev.points)...)
        scale = 0.9
    end
    
    progression = ProgressMeter.Progress(nsteps, desc="Loop in time: ", showspeed=true)
    animation = @animate for istep = 1:nsteps
        
        results.Eelec[istep], results.momentum[istep], results.Etot[istep] = PF_step!(particles, pmover; kernel=kernel)
        results.C[istep] = pmover.C[1]
        results.S[istep] = pmover.S[1]
        results.L²norm[istep] = sum(particles.wei.^2 ./ vec(weights))
        results.historyX[:, istep+1] .= particles.x
        results.historyV[:, istep+1] .= particles.v

        (results.max_area_triangles[istep], results.mean_area_triangles[istep], results.min_area_triangles[istep]) = triangle_area_triangulation(particles.x, particles.v)
        
        
        if istep % T == 0
            # p.wei .= nufft_interpolation(p.wei, p.x, p.v, init_pos, init_vel) .* weights
            particles.wei .= triangulation_interpolation(particles.wei ./ vec(weights), particles.x, particles.v, 
                                                            init_pos, init_vel, example)[1]
            particles.wei .*= vec(weights)
            particles.x .= init_pos
            particles.v .= init_vel
        end
        
        if plotting
            plot(particles.x, particles.v, seriestype=:scatter, zcolor=vec(particles.wei), 
                markersize=sqrt(600*600 / (nx*nv) / π) * scale, zticks=[], camera=(0, 90), 
                markerstrokecolor="white", markerstrokewidth=0, label="", c=:jet1,
                aspect_ratio=:equal, size=(600, 600), 
                title="t = $(@sprintf("%.3f",istep*dt))\nProgression: $(round(Int64,100*progression.counter / progression.n))%", titlefontsize=8, margin=5Plots.mm)
        end

        ProgressMeter.next!(progression)
    end when plotting
    if !plotting
        animation = nothing
    end
    
    results.Eelec .= sqrt.(results.Eelec)
    results.Etot .= sqrt.(results.Etot)
    results.L²norm .= sqrt.(results.L²norm)
    
    return results, animation
end

# Triangulation

In [ ]:
function triangle_area_triangulation(_pos, _vel)
    """
    Find the maximal area among all the triangles contained in the Delaunay triangulation

    Steps:
    1. Create the Delaunay triangulation
    2. For each triangle T, with sides of length a, b, c, use Heron's formula to compute the area 
            A = 1/4 √( (a+(b+c))(c - (a-b))(c+(a-b))(a+(b-c)) )
    """
    
    pointset = Triangulate.TriangulateIO()

    @views pointset.pointlist = vcat(_pos', _vel')
    (triangulation, _) = Triangulate.triangulate("Q", pointset);

    max_area = -1.
    mean_area = 0.
    min_area = Inf64

    @views @inbounds for (idxA, idxB, idxC) = eachcol(triangulation.trianglelist)
        
        A = triangulation.pointlist[:, idxA]
        B = triangulation.pointlist[:, idxB]
        C = triangulation.pointlist[:, idxC]

        a = √( (A[1] - B[1])*(A[1] - B[1]) + (A[2] - B[2])*(A[2] - B[2]))
        b = √( (A[1] - C[1])*(A[1] - C[1]) + (A[2] - C[2])*(A[2] - C[2]))
        c = √( (B[1] - C[1])*(B[1] - C[1]) + (B[2] - C[2])*(B[2] - C[2]))
        (c, b, a) = sqrt.(sort([a, b, c]))
        
        cur_area = 0.25*√( (a + (b + c))*(c - (a-b))*(c + (a-b))*(a + (b-c)) )
        (cur_area > max_area) && (max_area = cur_area)
        (cur_area < min_area) && (min_area = cur_area)
        mean_area += cur_area
    end

    return max_area, mean_area / size(triangulation.trianglelist, 2), min_area
end

In [ ]:
function triangulation_interpolation(_valsf, _pos, _vel, gridxoutput, gridvoutput, example; 
                                        returntriangulation=false)
    """
    Perform a linear interpolation of the data to the grid by using a Delaunay triangulation.

    Steps:
    1. Create the Delaunay triangulation
    2. For each point (x,v) in (gridxoutput, gridvoutput)
        a. Get the triangle T in which (x,v) lies, by iterating over all the triangles and testing each one of them
        b. From the vertices of T get a linear interpolation of the function at x, using barycentric coordinates
    """
    
    myoutput = similar(gridxoutput)

    pointset = Triangulate.TriangulateIO()

    @views pointset.pointlist = vcat(_pos', _vel')
    pointset.pointattributelist = _valsf'
    (triangulation, _) = Triangulate.triangulate("Q", pointset);

    indicestoreview = []

    # Interpolate using the triangulation
    @views @inbounds for part=eachindex(gridxoutput)
        myoutput[part] = triangleContainingPoint(triangulation, gridxoutput[part], gridvoutput[part])[1]
        myoutput[part] < 0 ? push!(indicestoreview, part) : nothing
    end

    # Perform rough approximation when the triangulation could not help us interpolate.
    # In this case we suppose the function is constant between the closest known value at the same velocity and the 
    # point at the desired location. 
    @views @inbounds for part=indicestoreview
        xline = findall((gridvoutput .== gridvoutput[part]).&&(myoutput .≥ 0)) 
        myoutput[part] = 0.0 # set value to 0.0 if nothing better can be done
        if isempty(xline)
            continue
        end
        x = gridxoutput[part]
        idxxminknown = minimum(xline)
        xminknown = gridxoutput[idxxminknown]
        idxxmaxknown = maximum(xline)
        xmaxknown = gridxoutput[idxxmaxknown]
        x > xmaxknown ? x -= example.L : nothing
        myoutput[part] = myoutput[idxxminknown] * (x-xmaxknown+example.L) / (xminknown - xmaxknown + example.L) +
                    myoutput[idxxmaxknown] * (1 - (x-xmaxknown+example.L) / (xminknown - xmaxknown + example.L))
    end 

    returntriangulation ? nothing : triangulation=nothing
    return vec(myoutput), triangulation
end


function triangleContainingPoint(triangulation, x, v)
    @views @inbounds for (idxA, idxB, idxC) = eachcol(triangulation.trianglelist)
        
        A = triangulation.pointlist[:, idxA]
        B = triangulation.pointlist[:, idxB]
        C = triangulation.pointlist[:, idxC]
        
        # if x is outside of the rectangle defined by [minX(A, B, C), maxX(A, B, C)],
        # or v is outside of the rectangle defined by [minV(A, B, C), maxV(A, B, C)],
        # then we don't have to do the computations
        if      ((x < A[1]) && (x < B[1]) && (x < C[1])) || ((x > A[1]) && (x > B[1]) && (x > C[1]))
            continue
        elseif  ((v < A[2]) && (v < B[2]) && (v < C[2])) || ((v > A[2]) && (v > B[2]) && (v > C[2]))
            continue
        end
        
        # Use barycentric coordinates:
        det = (A[1] - C[1])*(B[2] - C[2]) - (A[2] - C[2])*(B[1] - C[1])
        λ₁ = (x - C[1])*(B[2] - C[2]) + (v - C[2])*(C[1] - B[1])
        λ₂ = (x - C[1])*(C[2] - A[2]) + (v - C[2])*(A[1] - C[1])
        λ₁ /= det 
        λ₂ /= det
        
        if (λ₁≥0)&&(λ₂≥0)&&(λ₁+λ₂≤1)
            wA = triangulation.pointattributelist[1, idxA]
            wB = triangulation.pointattributelist[1, idxB]
            wC = triangulation.pointattributelist[1, idxC]
            return wA * λ₁ + wB * λ₂ + (1 - λ₁ - λ₂) * wC, (idxA, idxB, idxC)
        end
    end

    return -1., (-1, -1, -1)
end

# Inputs

In [ ]:
dev = CPU()

# example = example_landaudamping
# example = example_stronglandaudamping
example = example_twostreaminstability
# example = example_twostreaminstabilityalternativeformulation
# example = example_bumpontail
# example = example_stationarygaussian
# example = example_nonhomogeneousstationarysolution
# example = example_test

nstep = 1000
dt = 0.1

nxsl = 128
nvsl = 129
meshx = OneDGrid(dev, nxsl, 0, example.L);
meshv = OneDGrid(dev, nvsl, example.vmin, example.vmax);

quadX = RectangleRule
quadV = RectangleRule
nxpf = 128
nvpf = 129
quadrulex = quadX(nxpf, 0, example.L);
quadrulev = quadV(nvpf, example.vmin, example.vmax);
# # quadrulev = quadV(nv, μ - 5/√β, μ + 5/√β)
# quadrulev = quadV(nv, example.μ - 5/√example.β, example.μ + 5/√example.β)

# Simulations

In [ ]:
gsl = zeros(Complex{Float64}, (nxsl, nvsl));
@. gsl = example.f0.(meshx.points, meshv.points');
@time resSL, animSL = solve_SL!(nstep, dt, gsl, meshx, meshv, example.kx; plotting=false);

# PIC interpretation

In [ ]:
T = 100

nbparticles = nxpf*nvpf
x0_init = copy(vec(repeat(quadrulex.points, 1, nvpf)))
v0_init = copy(vec(repeat(quadrulev.points', nxpf, 1)))
x0 = copy(vec(repeat(quadrulex.points, 1, nvpf)))
v0 = copy(vec(repeat(quadrulev.points', nxpf, 1)))
weights = quadrulex.weights .* quadrulev.weights'
wei = vec(example.f0.(quadrulex.points, quadrulev.points') .* weights)
particles = Particles(x0, v0, wei, nbparticles);
@time resPF, animPF = solve_PF!(nstep, dt, particles, meshx, example, vec(weights) ; 
                                    plotting=false, kernel=kernel_poisson!, T=T);

In [ ]:
t = (1:nstep) .* dt


p1 = plot(legend=:bottomright, minorgrid=true, size=(600, 400))

# log(Energies)
plot!(p1, t .+ dt,   log10.(resPF.Eelec), label=L"\log_{10}(E_{elec, PF}),\quad dt="*"$(dt)")
plot!(p1, t .- dt/2, log10.(resSL.Eelec), label=L"\log_{10}(E_{elec, SL}),\quad dt="*"$(dt)", ls=:dash, lw=1)

title!(p1, example.longname * "\n($(quadX)($(nxpf)), $(quadV)($(nvpf)))", titlefontsize=8)
xlabel!(p1, "t (T=$(T))")
# xlabel!("time")

p2 = plot(resPF.max_area_triangles, xlabel="Step", ylabel="Triangle area", label="Max",
            margin=5Plots.mm, minorgrid=true)
p2 = plot!(p2, resPF.mean_area_triangles, label="Mean")
p2 = plot!(p2, resPF.min_area_triangles, label="Min")

plot(p1, p2, size=(1200, 600))

In [ ]:
savefig("../../../latex/imgs/png/area_triangles/$(example.shortname)_$(T).png")

In [ ]:
plot(vec(particles.x), vec(particles.v), seriestype=:scatter, markersize=sqrt(600*600 / (nxsl*nvsl) / π) * 0.5, camera=(0, 90), markerstrokecolor="white", markerstrokewidth=0, label="", zcolor=vec(particles.wei), c=:jet1, aspect_ratio=:equal, size=(600, 600), right_margin=5Plots.mm)